In [1]:
import pandas as pd

In [4]:
# Read in the dataset. 
healthcare_df = pd.read_csv("healthcare_dataset.csv")
healthcare_df

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.983364,146,Elective,2022-12-01,Aspirin,Inconclusive
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,Emergency,2023-06-15,Lipitor,Normal
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.896997,292,Emergency,2019-02-08,Lipitor,Normal
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.322092,480,Urgent,2020-05-03,Penicillin,Abnormal
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.344184,477,Urgent,2021-08-02,Paracetamol,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,James Hood,83,Male,A+,Obesity,2022-07-29,Samuel Moody,"Wood, Martin and Simmons",UnitedHealthcare,39606.840083,110,Elective,2022-08-02,Ibuprofen,Abnormal
9996,Stephanie Evans,47,Female,AB+,Arthritis,2022-01-06,Christopher Yates,Nash-Krueger,Blue Cross,5995.717488,244,Emergency,2022-01-29,Ibuprofen,Normal
9997,Christopher Martinez,54,Male,B-,Arthritis,2022-07-01,Robert Nicholson,Larson and Sons,Blue Cross,49559.202905,312,Elective,2022-07-15,Ibuprofen,Normal
9998,Amanda Duke,84,Male,A+,Arthritis,2020-02-06,Jamie Lewis,Wilson-Lyons,UnitedHealthcare,25236.344761,420,Urgent,2020-02-26,Penicillin,Normal


In [5]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")


In [6]:
import pandas as pd

# Assume df is your dataframe
# Convert dataframe to dictionary
data = healthcare_df.to_dict(orient='records')

# Choose a database and a collection
db = client["Healthcare_DB"]
collection = db["Healthcare"]

# Insert data into MongoDB
collection.insert_many(data)


InsertManyResult([ObjectId('660f5d9ea229b4fe879b1070'), ObjectId('660f5d9ea229b4fe879b1071'), ObjectId('660f5d9ea229b4fe879b1072'), ObjectId('660f5d9ea229b4fe879b1073'), ObjectId('660f5d9ea229b4fe879b1074'), ObjectId('660f5d9ea229b4fe879b1075'), ObjectId('660f5d9ea229b4fe879b1076'), ObjectId('660f5d9ea229b4fe879b1077'), ObjectId('660f5d9ea229b4fe879b1078'), ObjectId('660f5d9ea229b4fe879b1079'), ObjectId('660f5d9ea229b4fe879b107a'), ObjectId('660f5d9ea229b4fe879b107b'), ObjectId('660f5d9ea229b4fe879b107c'), ObjectId('660f5d9ea229b4fe879b107d'), ObjectId('660f5d9ea229b4fe879b107e'), ObjectId('660f5d9ea229b4fe879b107f'), ObjectId('660f5d9ea229b4fe879b1080'), ObjectId('660f5d9ea229b4fe879b1081'), ObjectId('660f5d9ea229b4fe879b1082'), ObjectId('660f5d9ea229b4fe879b1083'), ObjectId('660f5d9ea229b4fe879b1084'), ObjectId('660f5d9ea229b4fe879b1085'), ObjectId('660f5d9ea229b4fe879b1086'), ObjectId('660f5d9ea229b4fe879b1087'), ObjectId('660f5d9ea229b4fe879b1088'), ObjectId('660f5d9ea229b4fe879b10

In [10]:
print(client.list_database_names())

['Healthcare_DB', 'admin', 'classDB', 'config', 'dbName', 'fruits_db', 'local', 'met', 'travel_db', 'uk_food']


In [12]:
db = client['Healthcare_DB']

In [13]:

print(db.list_collection_names())

['Healthcare']


In [14]:
from pprint import pprint

In [15]:
document=db['Healthcare'].find_one()
pprint(document)

{'Admission Type': 'Elective',
 'Age': 81,
 'Billing Amount': 37490.98336352819,
 'Blood Type': 'O-',
 'Date of Admission': '2022-11-17',
 'Discharge Date': '2022-12-01',
 'Doctor': 'Patrick Parker',
 'Gender': 'Female',
 'Hospital': 'Wallace-Hamilton',
 'Insurance Provider': 'Medicare',
 'Medical Condition': 'Diabetes',
 'Medication': 'Aspirin',
 'Name': 'Tiffany Ramirez',
 'Room Number': 146,
 'Test Results': 'Inconclusive',
 '_id': ObjectId('660f5d9ea229b4fe879b1070')}
